In [1]:
from nwater.nw_hierarchy import nw_hierarchy_from_Guwahati, nw_hierarchy, nw_node, nw_instrument, nw_asset
from Guwahati import Guwahati

inst_info, asset_info = Guwahati.instrumentation_asset_info
root_node = nw_hierarchy_from_Guwahati(Guwahati.nodeinfo, inst_info)
nwh = nw_hierarchy(root_node=root_node)

nwh.pprint()

=== NW Hierarchy Pretty Print ===
📍 NW_root (ID: -1, Type: NW_root)
     🏢 Nijeshwari PWSS (ID: 97, Type: location)
          💧 Abstraction (ID: 99, Type: water_abstraction)
               🏔️ Source (ID: 100, Type: source_module)
                    🔧 Borewell level (ID: 69, Type: level, Primary: groundwater_level)
                         📦 D1004901122 (ID: 79, Product: FMX21)
                    🔧 Power Mains (ID: 70, Type: power, Primary: total_kwh)
                         📦 2407811548 (ID: 87, Product: Energy meter)
                    🔧 ABS-Pump (ID: 71, Type: pump, Primary: abs_pump_on)
                         📦 Guwahati_ABS-Pump01 (ID: 88, Product: Pump)
          🚰 Distribution (ID: 101, Type: water_distribution)
               🧽 Treated water (ID: 102, Type: disinfection_module)
                    🔧 Disinfect Transmitter (ID: 73, Type: undefined, Primary: None)
                         📦 L70E7D05G00 (ID: 80, Product: CM442)
                    🔧 Chlorine (ID: 75, Type: anal

In [ ]:
from nwater.nw_hierarchy import nw_node, nw_instrument, nw_asset, nw_hierarchy
from Guwahati import Guwahati   


def count_nodes_in_hierarchy(node, counts=None):
    """
    Count different types of nodes in the hierarchy.
    
    Args:
        node: The nw_node to count from
        counts: Dictionary to accumulate counts
    
    Returns:
        Dictionary with counts of each node type
    """
    if counts is None:
        counts = {
            'locations': 0,
            'applications': 0,
            'modules': 0,
            'instruments': 0,
            'assets': 0
        }
    
    # Count current node using isinstance checks
    if isinstance(node, nw_instrument):
        counts['instruments'] += 1
    elif isinstance(node, nw_asset):
        counts['assets'] += 1
    else:
        # This is a regular nw_node
        if node.type == 'location':
            counts['locations'] += 1
        elif node.type in ['water_abstraction', 'water_distribution', 'effluent_discharge']:
            counts['applications'] += 1
        elif 'module' in node.type:
            counts['modules'] += 1
    
    # Recursively count subnodes
    if hasattr(node, 'subnodes') and node.subnodes:
        for subnode in node.subnodes:
            count_nodes_in_hierarchy(subnode, counts)
    
    return counts

def pretty_print_nw_hierarchy(node, level=0):
    """
    Pretty print the NW hierarchy starting from a root node.
    
    Args:
        node: The nw_node to print (location, application, module, instrument, or asset)
        level: Current depth level for indentation (starts at 0)
    """
    # Create indentation (5 spaces per level)
    indent = "     " * level
    
    # Print current node with type information using isinstance checks
    if isinstance(node, nw_instrument):
        # This is an nw_instrument
        primary_key = node.primary_val_key or "None"
        print(f"{indent}🔧 {node.name} (ID: {node.id}, Type: {node.type}, Primary: {primary_key})")
    elif isinstance(node, nw_asset):
        # This is an nw_asset
        print(f"{indent}📦 {node.name} (ID: {node.id}, Product: {node.prod_code})")
    else:
        # This is a regular nw_node (location, application, module)
        type_emoji = {
            'location': '🏢',
            'water_abstraction': '💧',
            'water_distribution': '🚰',
            'effluent_discharge': '🌊',
            'source_module': '🏔️',
            'storage_module': '🏪',
            'disinfection_module': '🧽',
            'outlet_module': '🚪',
            'inlet_module': '🚪',
            'transfer_module': '🔄',
            'quality_control_module': '🔬'
        }.get(node.type, '📍')
        
        print(f"{indent}{type_emoji} {node.name} (ID: {node.id}, Type: {node.type})")
    
    # Recursively print all subnodes
    if hasattr(node, 'subnodes') and node.subnodes:
        for subnode in node.subnodes:
            pretty_print_nw_hierarchy(subnode, level + 1)

def print_hierarchy_summary(roots):
    """
    Print a summary of the entire hierarchy showing counts of each node type.
    
    Args:
        roots: List of root nodes to analyze
    """
    total_counts = {
        'locations': 0,
        'applications': 0,
        'modules': 0,
        'instruments': 0,
        'assets': 0
    }
    
    # Count nodes from all roots
    for root in roots:
        root_counts = count_nodes_in_hierarchy(root)
        for key in total_counts:
            total_counts[key] += root_counts[key]
    
    print("=" * 50)
    print("📊 HIERARCHY SUMMARY")
    print("=" * 50)
    print(f"🏢 Locations:      {total_counts['locations']:3d}")
    print(f"💧 Applications:   {total_counts['applications']:3d}")
    print(f"📦 Modules:        {total_counts['modules']:3d}")
    print(f"🔧 Instruments:    {total_counts['instruments']:3d}")
    print(f"📦 Assets:         {total_counts['assets']:3d}")
    print("-" * 50)
    total_nodes = sum(total_counts.values())
    print(f"📋 Total Nodes:    {total_nodes:3d}")
    print("=" * 50)

# Test the pretty print function with your hierarchy
print("=== NW Hierarchy Pretty Print ===")
pretty_print_nw_hierarchy(root)
print()

# Print summary using the root node
print_hierarchy_summary([root])

=== NW Hierarchy Pretty Print ===
📍 NW root (ID: -1, Type: NW root)
     🏢 Nijeshwari PWSS (ID: 97, Type: location)
          💧 Abstraction (ID: 99, Type: water_abstraction)
               🏔️ Source (ID: 100, Type: source_module)
                    🔧 Borewell level (ID: 69, Type: level, Primary: groundwater_level)
                         📦 D1004901122 (ID: 79, Product: FMX21)
                    🔧 Power Mains (ID: 70, Type: power, Primary: total_kwh)
                         📦 2407811548 (ID: 87, Product: Energy meter)
                    🔧 ABS-Pump (ID: 71, Type: pump, Primary: abs_pump_on)
                         📦 Guwahati_ABS-Pump01 (ID: 88, Product: Pump)
          🚰 Distribution (ID: 101, Type: water_distribution)
               🧽 Treated water (ID: 102, Type: disinfection_module)
                    🔧 Disinfect Transmitter (ID: 73, Type: undefined, Primary: None)
                         📦 L70E7D05G00 (ID: 80, Product: CM442)
                    🔧 Chlorine (ID: 75, Type: anal